In [18]:
import pandas as pd

data_address = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding = 'euc-kr')
data_address

,역명,주소,위도,경도
0,서울,서울특별시 중구 세종대로 지하2(남대문로 5가),37.553150,126.972533
1,시청,서울특별시 중구 세종대로 지하101(정동),37.563590,126.975407
2,종각,서울특별시 종로구 종로 지하55(종로1가),37.570203,126.983116
3,종로3가,서울특별시 종로구 종로 지하129(종로3가),37.570429,126.992095
4,종로5가,서울특별시 종로구 종로 지하216(종로5가),37.570971,127.001900
...,...,...,...,...
387,금릉,경기 파주시 금릉역로 85,37.751339,126.765410
388,금촌,경기 파주시 새꽃로 193,37.766162,126.774620
389,월롱,경기 파주시 월롱면 통일로 980,37.796215,126.792586
390,파주,경기 파주시 파주읍 주라위길 38,37.815366,126.792691


In [19]:
result = data_address[data_address['역명'].str.contains('\(.*\)', regex=True)]

# 결과 출력
print("[괄호가 포함된 역명 전체]")
print(result[['역명']])


[괄호가 포함된 역명 전체]
           역명
313  관악산(서울대)


<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_7682/2514821611.py:1: SyntaxWarning: invalid escape sequence '\('
  result = data_address[data_address['역명'].str.contains('\(.*\)', regex=True)]


In [20]:
import pandas as pd

data_address_latlng = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding = 'euc-kr')
data_address_latlng

,역명,주소,위도,경도
0,서울,서울특별시 중구 세종대로 지하2(남대문로 5가),37.553150,126.972533
1,시청,서울특별시 중구 세종대로 지하101(정동),37.563590,126.975407
2,종각,서울특별시 종로구 종로 지하55(종로1가),37.570203,126.983116
3,종로3가,서울특별시 종로구 종로 지하129(종로3가),37.570429,126.992095
4,종로5가,서울특별시 종로구 종로 지하216(종로5가),37.570971,127.001900
...,...,...,...,...
387,금릉,경기 파주시 금릉역로 85,37.751339,126.765410
388,금촌,경기 파주시 새꽃로 193,37.766162,126.774620
389,월롱,경기 파주시 월롱면 통일로 980,37.796215,126.792586
390,파주,경기 파주시 파주읍 주라위길 38,37.815366,126.792691


In [21]:
import folium
import pandas as pd

# CSV 파일에서 지하철 역 데이터 불러오기
data_address_latlng = pd.read_csv("../data/지하철_1_8호선 역주소(위경도) 좌표 정보.csv", encoding = 'euc-kr') # 파일명은 알맞게 수정

# 지도 중심 좌표 설정 (서울 시청 기준)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 각 지하철역에 대해 Circle 마커 생성
for _, row in data_address_latlng.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = str(row['역명'])[:2]  # 역 이름 앞 2글자
    folium.Circle(
        location=[lat, lon],
        radius=500,  # 반지름 (단위: meter)
        color='skyblue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.1,
        popup=station_name
    ).add_to(m)
m

# HTML로 저장 또는 노트북에서 직접 출력
#m.save("subway_map.html")


In [30]:
import pandas as pd
import folium

# 1. 파일 불러오기
coord_df = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding='euc-kr')

# 2. 필요 컬럼만 추출
coord_df = coord_df[['역명', '주소', '위도', '경도']]

# ✅ 3. NaN 값 제거 (주소, 위도, 경도 중 하나라도 없으면 제외)
coord_df = coord_df.dropna(subset=['주소', '위도', '경도'])

# ✅ 4. '주소'가 '서울특별시'로 시작하는 행만 필터링
coord_df = coord_df[coord_df['주소'].str.startswith('서울특별시', na=False)]

# 5. folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 6. Circle 마커 추가
for _, row in coord_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = str(row['역명'])[:2]  # 앞 두 글자만 표시
    folium.Circle(
        location=[lat, lon],
        radius=500,
        color='skyblue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.2,
        popup=station_name
    ).add_to(m)

# 7. 지도 출력 또는 저장
#m.save("서울특별시_지하철역_지도.html")
m


In [31]:
import pandas as pd
import re

# 1. 데이터 불러오기
df = pd.read_csv("../data/subway_complexity_toprank.csv", encoding='utf-8')

# 2. 가장 낮은 호선만 추출하는 함수
def get_lowest_line(line_str):
    numbers = re.findall(r'\d+', str(line_str))  # '1호선4호선' → ['1', '4']
    if not numbers:
        return '기타'
    min_num = min(map(int, numbers))
    return f"{min_num}호선"

# 3. 적용
df['대표호선'] = df['호선명'].apply(get_lowest_line)
df.to_csv("../data/subway_complexity_toprank_대표호선.csv")


In [32]:
top30_df.head(10)

,Unnamed: 0.1,Unnamed: 0,역명,조사연도,호선명,06시,07시,08시,09시,10시,...,20시,21시,22시,23시,환승여부,sum,morning,evening,대표호선,total_commute
0,0,0,잠실,4046,2호선8호선,19576,83632,214604,405802,322711,...,365864,397234,267171,205280,YY,5194890,704038,1346608,2호선,2050646
1,1,1,서울역,10115,1호선4호선경부선경의선공항철도 1호선,24098,97194,245019,460901,325796,...,295473,251044,203909,140926,YYYYY,4756511,803114,1217898,1호선,2021012
7,7,7,가산디지털단지,4046,7호선경부선,21491,54967,203562,543315,304900,...,199988,179158,93795,60974,YY,3145125,801844,906708,7호선,1708552
3,3,3,강남,2023,2호선,20720,65990,171696,333966,351250,...,311655,315962,225747,212947,Y,4331681,571652,1110558,2호선,1682210
2,2,2,홍대입구,6069,2호선경의선공항철도 1호선,37949,55719,110724,209480,199723,...,367369,363598,293882,277901,YYY,4625083,375923,1236319,1호선,1612242
5,5,5,선릉,4046,2호선분당선,12674,57975,167146,435733,343493,...,224871,222290,134275,112242,YY,3547166,660854,926236,2호선,1587090
6,6,6,사당,4046,2호선4호선,25240,94396,187945,257934,205831,...,267063,175264,185340,172660,YY,3433269,540275,886495,2호선,1426770
8,8,8,구로디지털단지,2023,2호선,35805,65580,200281,403443,245449,...,234387,126496,133662,115294,N,3004641,669304,743515,2호선,1412819
11,11,11,역삼,2023,2호선,10418,45182,149728,372619,323424,...,190120,181724,103221,72159,N,2851275,567529,719639,2호선,1287168
10,10,10,삼성,2023,2호선,9433,45184,136469,334978,242010,...,182040,225744,112010,71544,N,2871467,516631,769783,2호선,1286414


In [ ]:
import pandas as pd
import folium

# 1. 데이터 불러오기
df = pd.read_csv("../data/subway_complexity_toprank_대표호선.csv", encoding='utf-8')
coord_df = pd.read_csv("../data/지하철_1_8호선 역주소(위경도) 좌표 정보.csv", encoding='euc-kr')
addr_df = pd.read_csv("../data/지하철_역주소 및 전화번호.csv", encoding='euc-kr')

# 2. 주소 컬럼 정제
addr_df = addr_df[['호선', '역명', '도로명주소']]

# 3. 좌표 + 주소 병합
merged_df = pd.merge(coord_df, addr_df, on='역명', how='left')

# 4. 서울특별시만 필터링
filtered_df = merged_df[merged_df['도로명주소'].str.startswith('서울특별시', na=False)]

# 5. 출근+퇴근 합계 컬럼 추가
df['total_commute'] = df['morning'] + df['evening']

# 6. 출퇴근 합 기준 상위 30개 역 추출
top30_df = df.sort_values(by='total_commute', ascending=False).head(30)

# 7. 병합 (역명 기준)
final_df = pd.merge(top30_df, filtered_df, on='역명', how='inner')

# 8. folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 9. 마커 추가
for _, row in final_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = row['역명']
    morning = row['morning']
    evening = row['evening']
    total_commute = row['total_commute']
    line = row['대표호선']  # 새로 추가된 컬럼 사용

    # 색상 지정
    if total_commute >= 1200000:
        color = 'red'
    elif total_commute >= 1000000:
        color = 'orange'
    else:
        color = 'green'

    popup_text = f"""<b>{station_name} ({line})</b><br>
    출근: {morning:,}명<br>
    퇴근: {evening:,}명<br>
    합계: {total_commute:,}명"""

    folium.Circle(
        location=[lat, lon],
        radius=50,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.4,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# 10. 지도 저장 (선택)
#m.save("서울_출퇴근_혼잡도_top30_대표호선.html")

# 11. 지도 출력
m


In [33]:
import pandas as pd
import folium

# 1. 파일 불러오기
toprank_df = pd.read_csv("../data/subway_complexity_toprank_대표호선.csv", encoding='utf-8')
# 1. 파일 불러오기
coord_df = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding='euc-kr')

# 2. 필요 컬럼만 추출
coord_df = coord_df[['역명', '주소', '위도', '경도']]

# ✅ 3. NaN 값 제거 (주소, 위도, 경도 중 하나라도 없으면 제외)
coord_df = coord_df.dropna(subset=['주소', '위도', '경도'])

# ✅ 4. '주소'가 '서울특별시'로 시작하는 행만 필터링
coord_df = coord_df[coord_df['주소'].str.startswith('서울특별시', na=False)]

# 5. 출근 + 퇴근 합계 컬럼 추가
toprank_df['total_commute'] = toprank_df['morning'] + toprank_df['evening']

# 6. total_commute 기준 상위 30개 역 추출
top30_df = toprank_df.sort_values(by='total_commute', ascending=False).head(30)

# 7. 좌표와 병합 (역명 기준)
final_df = pd.merge(top30_df, coord_df, on='역명', how='inner')

# 8. 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 9. 마커 표시
for _, row in final_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = row['역명']
    morning = row['morning']
    evening = row['evening']
    total_commute = row['total_commute']
    
    # 색상 조건
    if total_commute >= 1200000:
        color = 'red'
    elif total_commute >= 1000000:
        color = 'orange'
    else:
        color = 'green'

    popup_text = f"""<b>{station_name}</b><br>
    출근: {morning:,}명<br>
    퇴근: {evening:,}명<br>
    합계: {total_commute:,}명"""

    folium.Circle(
        location=[lat, lon],
        radius=50,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.1,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# 10. 저장 (선택)
m.save("서울_출퇴근_혼잡도_top30.html")

# Jupyter에서 출력



In [ ]:
df = pd.read_csv("../data/지하철_역주소 및 전화번호.csv", encoding='euc-kr', index_col=0)
df['역명'] = df['역명'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df.to_csv("../data/지하철_역주소 및 전화번호_부기역명제거.csv", encoding='euc-kr')


In [ ]:
df = pd.read_csv("../data/지하철_역주소 및 전화번호_부기역명제거.csv", encoding='euc-kr', index_col=0)

df.loc[df['역명']=='서울', '역명'] = df.loc[df['역명']=='서울', '역명'].str.replace('서울', '서울역')

df.to_csv("../data/지하철_역주소 및 전화번호_부기역명제거.csv", encoding='euc-kr')
df

In [ ]:
import pandas as pd
import folium

# 1. 파일 불러오기
toprank_df = pd.read_csv("../data/subway_complexity_toprank_대표호선.csv", encoding='utf-8')
# 1. 파일 불러오기
coord_df = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding='euc-kr')

# ✅ 3. NaN 값 제거 (주소, 위도, 경도 중 하나라도 없으면 제외)
coord_df = coord_df.dropna(subset=['주소', '위도', '경도'])

# ✅ 4. '주소'가 '서울특별시'로 시작하는 행만 필터링
coord_df = coord_df[coord_df['주소'].str.startswith('서울특별시', na=False)]

# 5. 출근 + 퇴근 합계 컬럼 추가
toprank_df['total_commute'] = toprank_df['morning'] + toprank_df['evening']

# 6. total_commute 기준 상위 30개 역 추출
top30_df = toprank_df.sort_values(by='total_commute', ascending=False).head(30)

# 7. 좌표와 병합 (역명 기준)
final_df = pd.merge(top30_df, filtered_df, on='역명', how='inner')

# 8. 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 9. 마커 표시
for _, row in final_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = row['역명']
    morning = row['morning']
    evening = row['evening']
    total_commute = row['total_commute']
    
    # 색상 조건
    #if total_commute >= 1200000:
    #    color = 'red'
    #elif total_commute >= 1000000:
    #    color = 'orange'
    #else:
    #    color = 'green'

    popup_text = f"""<b>{station_name}</b><br>
    출근: {morning:,}명<br>
    퇴근: {evening:,}명<br>
    합계: {total_commute:,}명"""

    folium.Circle(
        location=[lat, lon],
        radius=600,
        color="red",
        fill=True,
        fill_color=color,
        fill_opacity=0.1,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# 10. 저장 (선택)
m.save("서울_출퇴근_혼잡도_top30_totalcommute.html")

# Jupyter에서 출력
m


In [ ]:


df = pd.read_csv("../data/지하철_1_8호선 역주소(위경도)_호선통합.csv", encoding='euc-kr', index_col=0)
df.loc[df['역명']=='서울', '역명'] = df.loc[df['역명']=='서울', '역명'].str.replace('서울', '서울역')
df.to_csv("../data/지하철_1_8호선 역주소(위경도)_호선통합.csv", encoding='euc-kr')
df

In [35]:
Jongno3ga = coord_df[coord_df['역명'] == '종로3가'][['역명', '위도', '경도']]

print(Jongno3ga)

     역명         위도          경도
3  종로3가  37.570429  126.992095


In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('../data/지하철_1_8호선 역주소(위경도) 좌표 정보.csv', encoding='euc-kr')  # 실제 파일명으로 수정하세요

# 호선에서 숫자만 추출 (예: '1호선' → 1)
df['호선번호'] = df['호선'].astype(str).str.extract('(\d+)').astype(int)

# 호선번호가 낮은 순으로 정렬 후, 역명별로 가장 낮은 호선의 행만 남김
representative_coords = (
    df.sort_values('호선번호')  # 낮은 호선이 우선
    .drop_duplicates(subset='역명', keep='first')  # 역명별 첫 번째 유지
    [['역명', '위도', '경도', '호선']]  # 필요한 열만
    .rename(columns={'호선': '대표호선'})  # 대표호선으로 열 이름 변경
)

# 원본 df에서 위도/경도 제거 후 병합 → 대표 위도/경도 반영
df_updated = df.drop(columns=['위도', '경도']).merge(representative_coords, on='역명', how='left')

# 역명별 하나만 남김 (중복 제거)
df_final = df_updated.drop_duplicates(subset='역명', keep='first')

# 저장
df_final.to_csv('../data/지하철_1_8호선 역주소(위경도)_호선통합.csv', index=False, encoding='euc-kr')


In [ ]:
df = pd.read_csv('../data/지하철_1_8호선 역주소(위경도)_호선통합.csv', encoding='euc-kr')
Jongno3ga2 = df[df['역명'] == '종로3가'][['역명', '위도', '경도']]

print(Jongno3ga2)